# Chattbottar - Kapitel 10 

I detta kodexempel ska vi få en grundläggande förståelse för hur chattbottar fungerar. Let's go! 

In [9]:
# pip install  google-genai
# pip install pypdf

In [3]:
import numpy as np
import google.generativeai as genai
from pypdf import PdfReader
from dotenv import load_dotenv
import os

# Använda en chattbott genom API
Vi kommer nu demonstrera hur vi kan ha en egen chattbott genom att använda en API-nyckel från Google i detta fall. Andra alternativ är t.ex. ChatGPT/OpenAI. 
Om du i verkligheten ska arbeta med chattbottar kommer du mest sannolikt att använda färdiga API. 

För att skapa en API-nyckel behöver du gå in på följande hemsids: https://aistudio.google.com/ och sen gå till "Create API key". Notera, du behöver registrera ditt bankkort för att få en gratis provperiod. Annars funkar det inte. Som vanlig praxis inom systemutveckling så delar du inte din privata API-nyckel eftersom då kan andra använda den vilket kostar pengar. Man kan använda "api_key = api_key=os.getenv("API_KEY")" för detta ändamål. I koden nedan **skriver jag explicit ut min API-nyckel som du dock inte kommer kuna använda (du behöver skapa en egen)**. 

In [14]:
load_dotenv()

api_key = os.getenv("API_KEY")

genai.configure(api_key=api_key)

# Create a model and generate content
model = genai.GenerativeModel("gemini-3-flash-preview")
response = model.generate_content("Skriv i 5 meningar vad som kännetecknar en skicklig programmerare.")

print(response.text)

Här är fem meningar som kännetecknar en skicklig programmerare:

1. En skicklig programmerare har en stark logisk förmåga och kan bryta ner komplexa problem i mindre, hanterbara delar.
2. Hen skriver ren och strukturerad kod som inte bara löser uppgiften, utan också är enkel för andra att förstå och underhålla.
3. En ständig nyfikenhet och vilja att lära sig nya tekniker är nödvändigt i en bransch som hela tiden utvecklas.
4. Utöver teknisk expertis krävs god kommunikationsförmåga för att kunna samarbeta effektivt i team och förstå användarnas behov.
5. Slutligen kännetecknas en duktig utvecklare av ett tålmodigt och noggrant arbetssätt, särskilt vid felsökning och optimering av system.


# Skapa en mycket enkel applikation
Vi kan också skapa en enkel applikation. Men lite kreativitet så inser man att det finns många möjligheter att bygga/utveckla sådant man är intresserad av.

In [20]:
print("*** Gemini chat ***")
print("Type <q> to exit chat.")

while True:
    prompt = input("User: ")
    if prompt == "q":
        break
    else:
        response = model.generate_content(
            model=genai.GenerativeModel("gemini-3-flash-preview"),
            contents=prompt
        )
        print("Gemini:", response.text)

*** Gemini chat ***
Type <q> to exit chat.


# Retrieval Augmented Generation (RAG)

## Läsa in PDF-fil

Vi börjar med att läsa in en PDF-fil som chattbotten kommer ge svar utifrån. 

In [22]:
reader = PdfReader("The_3_Kanto_Starters.pdf")

text = ""
for page in reader.pages:
    text += page.extract_text()

incorrect startxref pointer(1)
parsing for Object Streams


In [23]:
print(len(text))

807


In [24]:
print(text[17:550])

ers
The Kanto region introduced players to the Pokémon world with three iconic starter Pokémon. Each
represents a different type and playstyle, making the choice an important one for new trainers.
Bulbasaur ■
Bulbasaur is a Grass/Poison-type Pokémon known for its balanced stats and strong status moves. The
plant bulb on its back grows as it absorbs sunlight. Bulbasaur evolves into Ivysaur and then Venusaur.
Charmander ■
Charmander is a Fire-type Pokémon that excels in offensive power. The flame on its tail represents its
life f


## Chunking

### Fixed length-chunking

In [41]:
chunks = []
n = 500
overlap = 50
for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Antal chunks: {len(chunks)}.")

Antal chunks: 2.


In [42]:
print(chunks[0])

The 3 Kanto Starters
The Kanto region introduced players to the Pokémon world with three iconic starter Pokémon. Each
represents a different type and playstyle, making the choice an important one for new trainers.
Bulbasaur ■
Bulbasaur is a Grass/Poison-type Pokémon known for its balanced stats and strong status moves. The
plant bulb on its back grows as it absorbs sunlight. Bulbasaur evolves into Ivysaur and then Venusaur.
Charmander ■
Charmander is a Fire-type Pokémon that excels in offensive 


In [43]:
print(text[26:584])

anto region introduced players to the Pokémon world with three iconic starter Pokémon. Each
represents a different type and playstyle, making the choice an important one for new trainers.
Bulbasaur ■
Bulbasaur is a Grass/Poison-type Pokémon known for its balanced stats and strong status moves. The
plant bulb on its back grows as it absorbs sunlight. Bulbasaur evolves into Ivysaur and then Venusaur.
Charmander ■
Charmander is a Fire-type Pokémon that excels in offensive power. The flame on its tail represents its
life force. Charmander evolves into Char


## Embeddings
Se t.ex. s.321 i kursboken "Lär dig AI från grunden - Tillämpad maskininlärning med Python" för vad Embeddings innebär. I korthet, ord representeras med vektorer/siffror. 

In [44]:
def create_embeddings(text, embedding_model="models/text-embedding-004", task_type="retrieval_document"):
    """Create embeddings using Google's Generative AI"""
    result = genai.embed_content(
        model=embedding_model,
        content=text,
        task_type=task_type
    )
    return result

In [45]:
embeddings = create_embeddings(chunks)
len(embeddings['embedding'])

2

In [46]:
len(embeddings['embedding'])

embeddings['embedding'][0:10]

[[-0.02776055,
  -0.0020288746,
  -0.00034144378,
  -0.021767227,
  0.004281998,
  0.02508788,
  0.025097648,
  -0.02048101,
  -0.033283807,
  0.04049747,
  -0.0711511,
  0.0040550623,
  0.043484326,
  0.007921901,
  -0.03817788,
  -0.033158742,
  0.050904695,
  0.034559518,
  -0.078128725,
  0.034261145,
  -0.026563905,
  -0.005082299,
  -0.01848333,
  -0.025922216,
  0.011889414,
  -0.008202873,
  0.051103648,
  0.035761856,
  -0.0128277335,
  -0.012959791,
  -0.006338547,
  0.00876084,
  0.024397109,
  -0.045543406,
  -0.0040560765,
  -0.0024793344,
  0.013379501,
  0.025112808,
  0.026589803,
  -0.034934923,
  -0.053492595,
  0.033635218,
  0.0038425978,
  -0.033726867,
  -0.010320935,
  -0.012552402,
  -0.029808376,
  0.06162471,
  -0.02683285,
  0.038657334,
  -0.013087087,
  0.08152205,
  -0.07559114,
  0.015692778,
  -0.015764054,
  -0.0011591202,
  -0.03871346,
  0.01693279,
  0.031282812,
  -0.0035495106,
  -0.042804617,
  0.03296377,
  0.04009446,
  -0.04834485,
  0.00231620

## Semantisk sökning

In [47]:
def cosine_similarity(vec1, vec2):
    return (np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2)))

In [48]:
def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = create_embeddings(query)['embedding']
    similarity_scores = []
    
    for i, chunk_embedding in enumerate(embeddings['embedding']):
        similarity_score = cosine_similarity(query_embedding, chunk_embedding)
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    return [chunks[index] for index in top_indices]

In [49]:
fråga = "Vad är en squirtle för något?"
svar = semantic_search(fråga, chunks=chunks, embeddings=embeddings, k=1)
print(svar)

['r is a Fire-type Pokémon that excels in offensive power. The flame on its tail represents its\nlife force. Charmander evolves into Charmeleon and finally Charizard.\nSquirtle ■\nSquirtle is a Water-type Pokémon with strong defensive capabilities. It uses its shell for protection and\npowerful water attacks. Squirtle evolves into Wartortle and then Blastoise.\n']


## Generera bra svar med RAG

In [50]:
system_prompt = """Jag kommer ställa dig en fråga, och jag vill att du svarar
baserat bara på kontexten jag skickar med, och ingen annan information.
Om det inte finns nog med information i kontexten för att svara på frågan,
säg "Det vet jag inte". Försök inte att gissa.
Formulera dig enkelt och dela upp svaret i fina stycken. """

In [51]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks, embeddings))
    user_prompt = f"Frågan är {query}. Här är kontexten: {context}."
    return user_prompt

In [59]:
def generate_response(system_prompt, user_message, model_name="gemini-3-flash-preview"):
    model_instance = genai.GenerativeModel(
        model_name=model_name,
        system_instruction=system_prompt
    )
    response = model_instance.generate_content(
        generate_user_prompt(user_message)
    )
    return response

In [60]:
print(generate_response(system_prompt, "Vad är en Squirtle?").text)

Squirtle är en Pokémon av vattentyp som har starka defensiva förmågor. Den använder sitt skal för att skydda sig och för att utföra kraftfulla vattenattacker.

Den är en av de tre ikoniska start-Pokémon som introducerades i Kanto-regionen för nya tränare.

Squirtle utvecklas först till Wartortle och därefter till Blastoise.


In [61]:
fråga = "Vad är en bulbasaur?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Bulbasaur är en Pokémon av typen Grass/Poison. Den är känd för att ha balanserad statistik och starka förmågor som kan påverka motståndarens tillstånd.

På sin rygg har Bulbasaur en växtlök. Denna lök växer i takt med att den absorberar solljus.

Bulbasaur är en av de tre start-Pokémon som tränare kan välja i Kanto-regionen. Den utvecklas först till Ivysaur och slutligen till Venusaur.


In [68]:
fråga = "Gillar Antonio Squirtle?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Det vet jag inte.


# Evaluering

In [73]:
validation_data = [
{"question": "Vad heter eld pokemonen och dens 2 evolutioner?",
"ideal_answer": """Eldpokemonen heter Charmander. Dess tre evolutioner är Charmeleon och Charizard."""}
]

print(validation_data[0]["question"])
print()
print(validation_data[0]["ideal_answer"])

Vad heter eld pokemonen och dens 2 evolutioner?

Eldpokemonen heter Charmander. Dess tre evolutioner är Charmeleon och Charizard.


In [74]:
evaluation_system_prompt = """Du är ett intelligent utvärderingssystem vars uppgift är att utvärdera en AI-assistents svar. 
Om svaret är väldigt nära det önskade svaret, sätt poängen 1. Om svaret är felaktigt eller inte bra nog, sätt poängen 0.
Om svaret är delvis i linje med det önskade svaret, sätt poängen 0.5. Motivera kort varför du sätter den poäng du gör.
"""

In [75]:
query = validation_data[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data[0]['ideal_answer']}"""

# Note, we have created a "evaluation_system_prompt" and a "evaluation_prompt" that we use in our function "generate_response" that we created before. 
evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 1

Motivering: AI-assistenten svarar helt korrekt på frågan genom att identifiera både eld-Pokémonen (Charmander) och dess två evolutioner (Charmeleon och Charizard). Svaret är tydligt och i linje med det önskade svaret, även om det inte inkluderar råtexten från kontexten.


In [82]:
# Try change to "ideal_answer": "Java" and notice that if the user defines the wrong "ideal answer", then it gets weird. 
# In a wider context, how do you define ideal answers to questions that have no exact answer and who does this? 

validation_data_2 = [
{"question": "Vilken pokemon har Grass-type?",
"ideal_answer": "Charmander"}
]
print(validation_data_2[0]["question"])
print(validation_data_2[0]["ideal_answer"])

Vilken pokemon har Grass-type?
Charmander


In [83]:
query = validation_data_2[0]["question"]

response = generate_response(system_prompt, query)
print(response.text)

Bulbasaur är den pokemon som har Grass-type. Enligt texten är Bulbasaur en pokemon av typen Grass/Poison.

Bulbasaur utvecklas till Ivysaur och sedan till Venusaur. Den är känd för att ha balanserade egenskaper och en växtlök på ryggen som växer när den absorberar solljus.


In [84]:
evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data_2[0]['ideal_answer']}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 0

Motivering: AI-assistenten ger ett faktamässigt korrekt svar baserat på den tillhandahållna kontexten (där Bulbasaur beskrivs som Grass/Poison). Det önskade svaret anger dock "Charmander", vilket enligt kontexten är en Fire-type. Eftersom AI-assistentens svar (Bulbasaur) är en helt annan karaktär än det önskade svaret (Charmander), kan poängen inte bli 1, trots att AI-assistenten tekniskt sett har rätt och det önskade svaret verkar innehålla ett fel.
